In [1]:
# ==========================================
# 0. INSTALL DEPENDENCIES (Run this once)
# ==========================================
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps packaging ninja einops
!pip install --no-deps xformers trl peft accelerate bitsandbytes
!pip install datasets sentence-transformers faiss-cpu

import os
import json
import pandas as pd
from unsloth import FastLanguageModel
from google.colab import drive
import torch
import numpy as np
import re
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

# Use tqdm logic to determine best progress bar
try:
    from IPython import get_ipython
    if get_ipython():
        from tqdm.notebook import tqdm
    else:
        from tqdm import tqdm
except:
    from tqdm import tqdm


  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-2plbs4ok/unsloth_597bcf1cac6446e2b6e1cc5aa2e50e64
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-2plbs4ok/unsloth_597bcf1cac6446e2b6e1cc5aa2e50e64
  Resolved https://github.com/unslothai/unsloth.git to commit b96a04c17bc6bcb5522eafb17adc2b104be38f99
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 101.5 MB/s eta 0:00:00

In [3]:
# ==========================================
# 1. SETUP & CONFIGURATION
# ==========================================
# Mount Drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# Hugging Face Cache to Drive (Persistent)
DRIVE_CACHE = "/content/drive/MyDrive/LLM project/Cache/HF"
os.makedirs(DRIVE_CACHE, exist_ok=True)

os.environ["HF_HOME"] = f"{DRIVE_CACHE}/hf_home"
os.environ["HF_HUB_CACHE"] = f"{DRIVE_CACHE}/hf_hub"
os.environ["HF_DATASETS_CACHE"] = f"{DRIVE_CACHE}/datasets"
os.environ["TRANSFORMERS_CACHE"] = f"{DRIVE_CACHE}/transformers"
os.environ["SENTENCE_TRANSFORMERS_HOME"] = f"{DRIVE_CACHE}/sentence_transformers"

from huggingface_hub import login
# login(token="YOUR_HF_TOKEN")

# Paths
hf_model_id = "unsloth/Qwen2.5-7B-Instruct-bnb-4bit"
# Smart Load: Use local if available to avoid stuck downloads
# Using standard persistent drive path for models
drive_model_path = "/content/drive/MyDrive/LLM project/Models/Qwen2.5-7B-Instruct-bnb-4bit"

if os.path.exists(drive_model_path):
    print(f" Found local Qwen model at {drive_model_path}. Using it to load faster!")
    model_id = drive_model_path
else:
    print(f" Local model not found. Downloading {hf_model_id} from Hugging Face...")
    model_id = hf_model_id
    # We will save it later

input_path = "/content/drive/MyDrive/LLM project/DATA/task-a-en.tsv"
output_file = "/content/drive/MyDrive/LLM project/DATA/outputs_qwen_rag.jsonl"


 Local model not found. Downloading unsloth/Qwen2.5-7B-Instruct-bnb-4bit from Hugging Face...


In [4]:
# ==========================================
# 2. LOAD RAG SYSTEM (Retriever)
# ==========================================
class HFRetriever:
    def __init__(self, n_docs=25000):
        print(f"Loading embedded Wikipedia subset: {n_docs} docs ...")
        # Load pre-embedded dataset (fast)
        self.ds = load_dataset(
            "not-lain/wikipedia",
            revision="embedded",
            split=f"train[:{n_docs}]"
        )
        self.texts = [str(x) for x in self.ds["text"]]
        # Load embeddings into numpy (fast cosine sim)
        self.embs = np.array(self.ds["embeddings"], dtype=np.float32)

        # Normalize doc vectors once
        self.embs = self.embs / (np.linalg.norm(self.embs, axis=1, keepdims=True) + 1e-12)

        print("Loading query embedding model on CPU (Safe Mode)...")
        # Use the exact same model that was used to embed the dataset!
        self.encoder = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1", device="cpu")
        print("RAG Index Ready!")

    def retrieve(self, query, k=3, max_chars=1000):
        try:
            # Encode query
            q = self.encoder.encode([query]).astype(np.float32)
            q = q / (np.linalg.norm(q, axis=1, keepdims=True) + 1e-12)

            # Fast Cosine Similarity
            sims = self.embs @ q[0]
            top_idx = np.argsort(-sims)[:k]

            # Combine results
            combined_text = "\n\n".join([self.texts[i] for i in top_idx])
            return combined_text[:max_chars]
        except Exception as e:
            print(f"Retrieval failed: {e}")
            return ""

# Initialize Retriever (Fast Mode)
retriever = HFRetriever(n_docs=25000)


Loading embedded Wikipedia subset: 25000 docs ...


README.md:   0%|          | 0.00/417 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/49.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Loading query embedding model on CPU (Safe Mode)...


...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

RAG Index Ready!


In [5]:
# ==========================================
# 3. LOAD QWEN MODEL
# ==========================================
print(f"Loading Qwen from {model_id}...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    max_seq_length = 2048,
    load_in_4bit = True,
    dtype = None,
)
# Force model to CUDA explicitly to avoid device mismatch
# model = model.to("cuda") # CAUSES ERROR with 4-bit models
FastLanguageModel.for_inference(model)

# Save model if new
if model_id == hf_model_id and not os.path.exists(drive_model_path):
    print(f"💾 Saving model to Drive for future use: {drive_model_path}")
    model.save_pretrained(drive_model_path)
    tokenizer.save_pretrained(drive_model_path)

# Fix for Unsloth padding if missing
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def contains_chinese(text):
    return bool(re.search(r'[\u4e00-\u9fff]', text))


Loading Qwen from unsloth/Qwen2.5-7B-Instruct-bnb-4bit...
==((====))==  Unsloth 2026.1.2: Fast Qwen2 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

💾 Saving model to Drive for future use: /content/drive/MyDrive/LLM project/Models/Qwen2.5-7B-Instruct-bnb-4bit


In [6]:
# ==========================================
# 4. GENERATION LOOP
# ==========================================
# Read Input
df = pd.read_csv(input_path, sep='\t')
data = df.to_dict('records')

# Resume Check
processed_ids = set()
if os.path.exists(output_file):
    with open(output_file, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                saved_item = json.loads(line)
                processed_ids.add(saved_item['id'])
            except: pass
    print(f"Resuming... Found {len(processed_ids)} jokes.")

print("Starting RAG Generation with Qwen...")

for row in tqdm(data, desc="Generating Jokes"):
    current_id = row['id']
    if current_id in processed_ids: continue

    # Parse Input
    headline_val = str(row.get('headline', "-")).strip()
    w1_val = str(row.get('word1', "-")).strip()
    w2_val = str(row.get('word2', "-")).strip()

    # --- RAG RETRIEVAL ---
    if headline_val != "-" and headline_val != "" and headline_val.lower() != "nan":
        context = retriever.retrieve(headline_val)

        prompt_text = f"""### Instruction
You are a witty, cynical stand-up comedian. Write ONLY in English.
Use the following background information ONLY if it helps inspire a joke. Otherwise, ignore it.

Background Info:
{context}

Headline: "{headline_val}"

Task: Write EXACTLY ONE punchy joke (1-2 sentences) about this headline.
- Be clever, cynical, or ironic.
- Do NOT explain the joke.
- Output ONLY the joke.
- Output MUST be in English.

### Response
Joke:"""
        input_type = "headline"
        input_content = headline_val

    else:
        real_w1 = w1_val if w1_val != "-" else "something"
        real_w2 = w2_val if w2_val != "-" else "random"
        query = f"{real_w1} {real_w2}"
        context = retriever.retrieve(query)

        prompt_text = f"""### Instruction
You are a witty, cynical stand-up comedian. Write ONLY in English.
Use the following background information ONLY if it helps inspire a joke. Otherwise, ignore it.

Background Info:
{context}

Task: Write EXACTLY ONE punchy joke (1-2 sentences) connecting: "{real_w1}" and "{real_w2}".
- Be clever, cynical, or ironic.
- Do NOT explain the joke.
- Output ONLY the joke.
- Output MUST be in English.

### Response
Joke:"""
        input_type = "words"
        input_content = f"{real_w1}, {real_w2}"

    # --- RETRY LOOP FOR QWEN ENGLISH CHECK ---
    max_retries = 3
    final_joke = None

    for attempt in range(max_retries):
        # Apply Chat Template
        inputs = tokenizer.apply_chat_template(
            [{"role": "user", "content": prompt_text}],
            tokenize = True,
            add_generation_prompt = True,
            return_tensors = "pt"
        ).to("cuda")

        # Generate (Explicitly passing input_ids and attention_mask to avoid device errors)
        with torch.inference_mode(): # Memory optimization
            outputs = model.generate(
                input_ids=inputs,
                attention_mask=torch.ones_like(inputs), # Safe mask
                do_sample = True,
                max_new_tokens = 64, # Optimized
                temperature = 0.9,
                top_p = 0.9,
                repetition_penalty = 1.2,
                pad_token_id = tokenizer.eos_token_id
            )

        raw_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Robust Parsing
        if "Joke:" in raw_text:
            temp_joke = raw_text.split("Joke:")[-1]
        elif "assistant" in raw_text:
            temp_joke = raw_text.split("assistant")[-1]
        elif "Response" in raw_text:
            temp_joke = raw_text.split("Response")[-1]
        else:
            temp_joke = raw_text

        # Cleanup Artifacts
        temp_joke = temp_joke.replace("assistant", "")
        temp_joke = temp_joke.strip()
        temp_joke = temp_joke.split("\n\n")[0]

        # Remove leading/trailing quotes if any
        if temp_joke.startswith('"') and temp_joke.endswith('"'):
            temp_joke = temp_joke[1:-1]
        elif temp_joke.startswith("'") and temp_joke.endswith("'"):
            temp_joke = temp_joke[1:-1]

        temp_joke = temp_joke.strip()

        if contains_chinese(temp_joke):
            # print(f"ID {current_id} (Attempt {attempt+1}): Chinese detected. Retrying...")
            continue
        else:
            final_joke = temp_joke
            break

    if final_joke is None:
        final_joke = "ERROR: GENERATION FAILED (CHINESE DETECTED)"

    # Clean Llama/Qwen artifacts
    final_joke = final_joke.replace("assistant\n", "").strip()

    # Save
    result_entry = {
        "id": current_id,
        "type": input_type,
        "input_original": input_content,
        "retrieved_context": context,
        "prompt_used": prompt_text,
        "generated_joke": final_joke
    }

    with open(output_file, "a", encoding='utf-8') as f:
        f.write(json.dumps(result_entry, ensure_ascii=False) + "\n")

    # Periodic Cache Clear
    if len(processed_ids) % 50 == 0:
        torch.cuda.empty_cache()

print("Finished RAG Generation!")


Starting RAG Generation with Qwen...


Generating Jokes:   0%|          | 0/1200 [00:00<?, ?it/s]

Finished RAG Generation!
